<style type="text/css">
.tf  {border-collapse:collapse;border-spacing:0;width:100%}
.tf td{border-color:black;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;
  overflow:hidden;padding:10px 5px;word-break:normal;}
.tf th{border-color:black;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;
  font-weight:normal;overflow:hidden;padding:10px 5px;word-break:normal;}
.tf .tg-0pky{border-color:inherit;text-align:left;vertical-align:top}
.col1 { width: 20%;}
.col2 { width: 80%;}
</style>
<table class="tf">
<thead>
  <tr>
    <th class="tg-0pky col1">Name</th>
    <th class="tg-0pky col2">Hatim Sawai</th>
  </tr>
</thead>
<tbody>
  <tr>
    <td class="tg-0pky col1">UID No.</td>
    <td class="tg-0pky col2">2021300108</td>
  </tr>
  <tr>
    <td class="tg-0pky col1">Experiment No.</td>
    <td class="tg-0pky col2">7</td>
  </tr>
</tbody>
</table>

## <p style="text-align:center;font-weight:500;">Experiment 7</p>
<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;width:100%}
.tg td{border-color:black;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;
  overflow:hidden;padding:10px 5px;word-break:normal;}
.tg th{border-color:black;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;
  font-weight:normal;overflow:hidden;padding:10px 10px;word-break:normal;}
.tg .tg-0pky{border-color:inherit;text-align:left;vertical-align:top;}
.col1 { width: 20%;}
.col2 { width: 80%;}
</style>
<table class="tg">
<thead>
  <tr>
    <th class="tg-0pky col1">Aim</th>
    <th class="tg-0pky col2">Perform chunking by analyzing the importance of selecting proper features for training a model and size of training.</th>
  </tr>
</thead>
</table> 

### 1. Installation of NLTK and downloading the required corpus

In [3]:
import re
import warnings
import nltk
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.chunk import RegexpParser
from prettytable import PrettyTable
warnings.filterwarnings('ignore')

### 2. Loading the corpus and preprocessing

In [4]:
# load csv
df = pd.read_csv('../dataset/exp5.csv', encoding='isO-8859-1')
df1 = df[df['Sentence #'].notna()]
print("There are",df1['Sentence #'].iloc[-1].split()[-1],"sentences in the dataset")
df.drop(['Sentence #', 'Tag'], axis=1, inplace=True)
df.head()

There are 47959 sentences in the dataset


,Word,POS
0,Thousands,NNS
1,of,IN
2,demonstrators,NNS
3,have,VBP
4,marched,VBN


In [5]:
def preprocess(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text) # remove punctuation
    text = text.replace("\n", " ") # remove \n
    text = re.sub(r'\W', ' ', text)  # Remove non-word characters
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra whitespaces
    text = re.sub(r'\d', '', text)  # Remove digits
    return text

In [6]:
phrase_mapping = {
    'NP: {<DT>?<JJ>*<NN>}': 'noun phrase',
    'PP: {<IN><NP>}': 'prepositional phrase',
    'VP: {<VB.*><NP|PP|CLAUSE>+$}': 'verb phrase'
}

### 3. Chunking

In [6]:
# Sample text corpus (you can replace this with your own text corpus)
text_corpus = """
The quick brown fox jumps over the lazy dog. 
John likes eating pizza with his friends on Fridays.
She sings beautifully in the choir.
"""

sentences = sent_tokenize(text_corpus)

In [20]:
# Function to perform chunking using regular expressions
def chunk_with_regex(sentence):
    grammar = r"""
        NP: {<DT>?<JJ>*<NN>}   # Chunk sequences of DT, JJ, NN
        PP: {<IN><NP>}         # Chunk prepositions followed by NP
        VP: {<VB.*><NP|PP>*}   # Chunk verbs followed by NP or PP
        ADJP: {<JJ>+}          # Chunk sequences of JJ
        ADVP: {<RB.*>}         # Chunk adverbs
    """
    parser = RegexpParser(grammar)
    parsed_sentence = parser.parse(sentence)
    return parsed_sentence

In [19]:
# Function to perform chunking using NLTK library
def chunk_with_nltk(sentence):
    words = word_tokenize(sentence)
    tagged_words = nltk.pos_tag(words)
    grammar = r"""
        NP: {<DT>?<JJ>*<NN>}    # Chunk sequences of DT, JJ, NN
        PP: {<IN><NP>}          # Chunk prepositions followed by NP
        VP: {<VB.*><NP|PP>*}    # Chunk verbs followed by NP or PP
        ADJP: {<JJ>+}           # Chunk sequences of JJ
        ADVP: {<RB.*>}          # Chunk adverbs
    """
    parser = nltk.RegexpParser(grammar)
    parsed_sentence = parser.parse(tagged_words)
    return parsed_sentence

In [21]:
# Perform chunking using regular expressions
# Initialize PrettyTable
chunk_table = PrettyTable(["Tag", "Phrase"])
for sentence in sentences:
    parsed_sentence = chunk_with_regex(nltk.pos_tag(word_tokenize(sentence)))
    for subtree in parsed_sentence.subtrees():
        if subtree.label() in ['NP', 'PP', 'VP', 'ADJP', 'ADVP']:
            chunk_table.add_row([subtree.label(), " ".join(word for word, tag in subtree.leaves())])

# Print the table
print(chunk_table)

+------+-------------------------+
| Tag  |          Phrase         |
+------+-------------------------+
|  NP  |     The quick brown     |
|  NP  |           fox           |
|  VP  | jumps over the lazy dog |
|  PP  |    over the lazy dog    |
|  NP  |       the lazy dog      |
|  VP  |          likes          |
|  VP  |       eating pizza      |
|  NP  |          pizza          |
|  VP  |          sings          |
| ADVP |       beautifully       |
|  PP  |       in the choir      |
|  NP  |        the choir        |
+------+-------------------------+


In [10]:
# Perform chunking using NLTK library
print("\nChunking using NLTK library:")
for sentence in sentences:
    parsed_sentence = chunk_with_nltk(sentence)
    print(parsed_sentence)


Chunking using NLTK library:
(S
  (NP The/DT quick/JJ brown/NN)
  (NP fox/NN)
  (VP jumps/VBZ (PP over/IN (NP the/DT lazy/JJ dog/NN)))
  ./.)
(S
  John/NNP
  (VP likes/VBZ)
  (VP eating/VBG (NP pizza/NN))
  with/IN
  his/PRP$
  friends/NNS
  on/IN
  Fridays/NNS
  ./.)
(S
  She/PRP
  (VP sings/VBZ)
  (ADVP beautifully/RB)
  (PP in/IN (NP the/DT choir/NN))
  ./.)


### 4. Calculating Emission & Transition Probabilities

### 5. Predicting POS tags for a given sentence

### 6. Curiosity Questions

<p style="font-weight:500;">Q1. List a few ways for tagging parts of speech?</p>
Ans: There are several ways to tag parts of speech. Some of them are:
- Rule-based tagging  
<br>
- Stochastic tagging  
<br>
- Transformation-based tagging
<br>
- Hidden Markov Models
<br>
- Maximum Entropy Models
<br>
- Deep Learning Models

<p style="font-weight:500;">Q2. How do you find the most probable sequence of POS tags from a sequence of text?</p>
Ans: The most probable sequence of POS tags from a sequence of text can be found using Hidden Markov Models. The Viterbi algorithm is used to find the most probable sequence of POS tags from a sequence of text.

<p style="font-weight:500;">Q3. Differentiate between Markov chain and Markov model?</p>
Ans: Markov chain is a stochastic model describing a sequence of possible events in which the probability of each event depends only on the state attained in the previous event. Markov model is a model that assumes that the future state depends only on the current state and not on the sequence of events that preceded it. Markov chain is a special case of Markov model.

<p style="font-weight:500;">Q4. How you can identify whether a system follows a Markov Process?</p>
Ans: A system follows a Markov Process if it satisfies the Markov property. The Markov property states that the future state of the system depends only on the current state and not on the sequence of events that preceded it. If the system satisfies the Markov property, then it follows a Markov Process.

<p style="font-weight:500;">Q5.  Explain the use of Markov Chains in text generation algorithms.</p>
Ans: Markov Chains are used in text generation algorithms to generate text that is similar to the input text. Markov Chains are used to model the probability of the next word in a sequence of words given the current word. This probability is used to generate the next word in the sequence. Markov Chains are used to generate text that is similar to the input text by sampling the next word from the probability distribution of the next word given the current word.

### 6. Conclusion
In this experiment we learned about Hidden Markov Models and how to implement a Hidden Markov Model for tagging Parts of Speech. We also learned how to calculate the emission and transition matrix for tagging Parts of Speech using Hidden Markov Model. We also learned how to find the POS tag of a given sentence using Hidden Markov Model.